# 7. Data Enrichment 

Given the tags present in the data entries with the field 'malware'.
If the field 'malware' is NaN, i have use both the TAGS and the MALWARES dataset of the corresponding data entry to predict the value. 

1. I have applied similarity measures ( jaro wrinkler similarity index ) to the malwares and group the similiar malwares ( denoting the ALIAS dataset )

2. I have applied similarity measures ( jaro wrinkler similarity index ) between tags and the resulting ALIAS dataset

N.B. I note that in tags can be introduced multiple malwares so i need to find a criteria to choose or to consider the relationship ( entry, malware ) as 1:N


In [1]:
from datetime import datetime
import os
import time
import pandas as pd 
import requests
import urllib3
import json
import sys
import numpy as np
import math

pd.options.display.max_colwidth = 1000

In [2]:
file = r'C:\Users\leona\OneDrive\Desktop\Tesi\Pipeline\Datasets\Pipeline\DataIntegration\Entries.csv'


## Entries

In [3]:
df = pd.read_csv(file, low_memory = False)
df

,ID_ENTRY,first_seen,ioc,status,last_seen,threat_type,reference,reporter,source,ioc_type,malware,hostname,as_number,as_name,country
0,2711900.0,2023-09-15 15:17:09,http://1.32.53.97:45188/.i,online,2023-09-15 15:17:09,payload_delivery,https://urlhaus.abuse.ch/url/2711900/,geenensp,URL Haus,url,NaN,NaN,NaN,NaN,NaN
1,2711899.0,2023-09-15 15:11:08,https://kld.workout.oystergardener.net/editContent,offline,2023-09-15 15:11:08,payload_delivery,https://urlhaus.abuse.ch/url/2711899/,Cryptolaemus1,URL Haus,url,NaN,NaN,NaN,NaN,NaN
2,2711898.0,2023-09-15 15:03:04,http://178.141.204.195:38244/Mozi.m,offline,2023-09-15 15:03:04,payload_delivery,https://urlhaus.abuse.ch/url/2711898/,Gandylyan1,URL Haus,url,NaN,NaN,NaN,NaN,NaN
3,2711897.0,2023-09-15 13:35:07,https://github.com/hoter666/8888/blob/main/Banana.exe,online,2023-09-15 15:04:29,payload_delivery,https://urlhaus.abuse.ch/url/2711897/,Casperinous,URL Haus,url,NaN,NaN,NaN,NaN,NaN
4,2711896.0,2023-09-15 13:33:07,http://mail.treeoflifeadventures.com/wp-content/plugins/70d5e28f51c1438d94e3e6dc84b95311/xt/mmd/shell/UMLrjk2KC4fpJg5.exe,offline,2023-09-15 13:56:00,payload_delivery,https://urlhaus.abuse.ch/url/2711896/,James_inthe_box,URL Haus,url,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15580,NaN,2023-08-24 23:29:43,41.97.47.7:443,offline,2023-08-25,NaN,NaN,NaN,Feodo Tracker,ip:port,QakBot,NaN,36947.0,ALGTEL-AS,ALGERIA
15581,NaN,2023-08-25 01:29:36,92.9.44.234:2222,offline,2023-08-25,NaN,NaN,NaN,Feodo Tracker,ip:port,QakBot,host-92-9-44-234.as13285.net,13285.0,OPALTELECOM-AS TalkTalk Communications Limited,GREAT BRITAIN
15582,NaN,2023-08-25 10:08:52,78.152.198.132:443,offline,2023-08-25,NaN,NaN,NaN,Feodo Tracker,ip:port,QakBot,NaN,15502.0,VODAFONE-IRELAND-ASN,IRELAND
15583,NaN,2023-08-25 11:24:40,113.193.95.229:443,offline,2023-08-25,NaN,NaN,NaN,Feodo Tracker,ip:port,QakBot,NaN,45528.0,TIKONAIN-AS Tikona Infinet Ltd.,INDIA


#### Data Cleaning 

### Malwares

In [14]:
file_malwares = r'C:\Users\leona\OneDrive\Desktop\Tesi\Pipeline\Datasets\Pipeline\DataIntegration\Malwares.csv'

malwares_table = pd.read_csv(file_malwares, low_memory = False)
malwares_table.drop

,malware,alias
0,Cobalt Strike,Agentemis
1,Nanocore RAT,Nancrat
2,Remcos,RemcosRAT
3,Loki Password Stealer (PWS),Burkina
4,DCRat,DarkCrystal RAT
...,...,...
173,Ave Maria,NaN
174,Houdini,dinihou
175,Houdini,dunihi
176,QakBot,NaN


In [36]:
def prepare_for_similarity_comparison(df, col_i, col_f):
    
    for i in range(len(df)):
        
        value = df.at[i,col_i]
        if (not isinstance(value, float) or not math.isnan(value)):    
            
            value = ' ' + value + ' ' 
            value = value.lower() 
            value = value.replace('.', ' ').replace('-', ' ').replace('[' , ' ').replace( ']', ' ').replace('(' , ' ').replace( ')', ' ').replace('\\',' ').replace( ',', ' ').replace( '_', ' ') 
            value = ''.join(char for char in value if not char.isdigit())
            value = value.strip() 
        
        df.loc[i, col_f] = value
            
    return df

In [37]:
malwares_table['type'] = 'unknown'
malwares_table.sort_values('malware')

,malware,alias,type
141,Agent Tesla,Negasteal,unknown
159,Agent Tesla,NaN,unknown
85,Agent Tesla,AgentTesla,unknown
17,Agent Tesla,AgenTesla,unknown
80,Alien,NaN,unknown
...,...,...,...
60,XWorm,NaN,unknown
43,pupy,Patpoopy,unknown
111,pupy,NaN,unknown
9,NaN,NaN,unknown


In [38]:
alias    = pd.DataFrame(malwares_table[malwares_table['alias'].notna()]['alias'].drop_duplicates()).reset_index(drop=True)
malwares = pd.DataFrame(malwares_table[malwares_table['malware'].notna()]['malware'].drop_duplicates()).reset_index(drop=True)

In [39]:
alias

,alias
0,Agentemis
1,Nancrat
2,RemcosRAT
3,Burkina
4,DarkCrystal RAT
...,...
71,Smoke Loader
72,avemaria
73,WSHRAT
74,dinihou


In [40]:
malwares

,malware
0,Cobalt Strike
1,Nanocore RAT
2,Remcos
3,Loki Password Stealer (PWS)
4,DCRat
...,...
64,SectopRAT
65,Ghost RAT
66,WhiteSnake Stealer
67,QakBot


### Tags

In [41]:
file_tags = r'C:\Users\leona\OneDrive\Desktop\Tesi\Pipeline\Datasets\Pipeline\DataIntegration\Tags.csv'

tags_table = pd.read_csv(file_tags, low_memory = False)
tags_table

,ID_ENTRY,tag,source
0,2704827,SocGholish,URL Haus
1,2704829,Mozi,URL Haus
2,2704830,Mozi,URL Haus
3,2704831,Mozi,URL Haus
4,2704832,Mozi,URL Haus
...,...,...,...
40137,1163859,RAT,Threat Fox
40138,1163860,RAT,Threat Fox
40139,1163860,NanoCore,Threat Fox
40140,1163861,RemcosRAT,Threat Fox


In [42]:
tags = pd.DataFrame(tags_table[tags_table['tag'].notna()]['tag'].drop_duplicates()).reset_index(drop=True)

In [43]:
tags

,tag
0,SocGholish
1,Mozi
2,elf
3,mirai
4,remcos
...,...
516,Private Layer INC
517,CrimsonRAT
518,CANTV Servicios Venezuela
519,PEG-TY


In [45]:
alias    = prepare_for_similarity_comparison(alias, 'alias', 'alias_to_compare')
malwares = prepare_for_similarity_comparison(malwares, 'malware', 'malware_to_compare')
tags     = prepare_for_similarity_comparison(tags, 'tag', 'tag_to_compare')

In [46]:
tags

,tag,tag_to_compare
0,SocGholish,socgholish
1,Mozi,mozi
2,elf,elf
3,mirai,mirai
4,remcos,remcos
...,...,...
516,Private Layer INC,private layer inc
517,CrimsonRAT,crimsonrat
518,CANTV Servicios Venezuela,cantv servicios venezuela
519,PEG-TY,peg ty


In [62]:
malwares

,malware,malware_to_compare
0,Cobalt Strike,cobalt strike
1,Nanocore RAT,nanocore rat
2,Remcos,remcos
3,Loki Password Stealer (PWS),loki password stealer pws
4,DCRat,dcrat
...,...,...
64,SectopRAT,sectoprat
65,Ghost RAT,ghost rat
66,WhiteSnake Stealer,whitesnake stealer
67,QakBot,qakbot


In [63]:
alias

,alias,alias_to_compare
0,Agentemis,agentemis
1,Nancrat,nancrat
2,RemcosRAT,remcosrat
3,Burkina,burkina
4,DarkCrystal RAT,darkcrystal rat
...,...,...
71,Smoke Loader,smoke loader
72,avemaria,avemaria
73,WSHRAT,wshrat
74,dinihou,dinihou


# Data Enrichment



### Malwares - Data Enrichment

Enrich the MALWARES dataset by using the TAGS dataset; We search in the tags if there are correspondences between alias, malwares and tags. If there are, they ate inserted in the Malwares dataset.

In [48]:
unique_alias = list(alias[alias['alias_to_compare'].notna()]['alias'].unique())
unique_malwares = list (malwares[malwares['malware_to_compare'].notna()]['malware_to_compare'].unique())
unique_tags     = list (tags[tags['tag_to_compare'].notna()]['tag_to_compare'].unique())

unique_columns = list(set(unique_alias + unique_malwares))

In [49]:
len(unique_alias)

76

In [50]:
len(unique_malwares)

69

In [51]:
len(unique_tags)

408

In [52]:
len(unique_columns)

145

#### Jaro-Wrinkler Similarity index

In [53]:
# Jaro Winkler Similarity
def jaro_Winkler(s1, s2) :
 
    jaro_dist = jaro_distance(s1, s2);
 
    # If the jaro Similarity is above a threshold
    if (jaro_dist > 0.7) :
 
        # Find the length of common prefix
        prefix = 0;
 
        for i in range(min(len(s1), len(s2))) :
         
            # If the characters match
            if (s1[i] == s2[i]) :
                prefix += 1;
 
            # Else break
            else :
                break;
 
        # Maximum of 4 characters are allowed in prefix
        prefix = min(4, prefix);
 
        # Calculate jaro winkler Similarity
        jaro_dist += 0.1 * prefix * (1 - jaro_dist);
 
    return jaro_dist;


def jaro_distance(s1, s2) :
 
    # If the strings are equal
    if (s1 == s2) :
        return 1.0;
 
    # Length of two strings
    len1 = len(s1);
    len2 = len(s2);
 
    if (len1 == 0 or len2 == 0) :
        return 0.0;
 
    # Maximum distance upto which matching
    # is allowed
    max_dist = (max(len(s1), len(s2)) // 2 ) - 1;
 
    # Count of matches
    match = 0;
 
    # Hash for matches
    hash_s1 = [0] * len(s1) ;
    hash_s2 = [0] * len(s2) ;
 
    # Traverse through the first string
    for i in range(len1) :
 
        # Check if there is any matches
        for j in range( max(0, i - max_dist),
                    min(len2, i + max_dist + 1)) :
             
            # If there is a match
            if (s1[i] == s2[j] and hash_s2[j] == 0) :
                hash_s1[i] = 1;
                hash_s2[j] = 1;
                match += 1;
                break;
         
    # If there is no match
    if (match == 0) :
        return 0.0;
 
    # Number of transpositions
    t = 0;
 
    point = 0;
 
    # Count number of occurrences
    # where two characters match but
    # there is a third matched character
    # in between the indices
    for i in range(len1) :
        if (hash_s1[i]) :
 
            # Find the next matched character
            # in second string
            while (hash_s2[point] == 0) :
                point += 1;
 
            if (s1[i] != s2[point]) :
                point += 1;
                t += 1;
            else :
                point += 1;
                 
        t /= 2;
 
    # Return the Jaro Similarity
    return ((match / len1 + match / len2 +
            (match - t) / match ) / 3.0);

In [54]:
def compute_matrix(lista_riga, lista_colonna):
    
    df = pd.DataFrame()
    
    df.index = lista_riga
    
    df.rename(columns=lista_colonna, inplace=True)  
    
    for i in range(len(lista_riga)):
        for j in range(len(lista_colonna)):
            
            string1 = lista_riga[i]
            string2 = lista_colonna[j]
            
            jaro_wrinkler = jaro_Winkler(string1,string2)
            
            df.loc[string1, string2] = jaro_wrinkler
            
    return df
            
            

In [55]:
jaro_wrinkler_matrix = compute_matrix( unique_tags, unique_columns)
jaro_wrinkler_matrix

C:\Users\leona\AppData\Local\Temp\ipykernel_848\2389614628.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[string1, string2] = jaro_wrinkler
C:\Users\leona\AppData\Local\Temp\ipykernel_848\2389614628.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[string1, string2] = jaro_wrinkler
C:\Users\leona\AppData\Local\Temp\ipykernel_848\2389614628.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joinin

,kimsuky,IceID,Warzone RAT,Socmer,DarkCrystal RAT,astaroth,hydra,loda,Hworm,Negasteal,...,Patpoopy,AgentTesla,havoc,Dofoil,Smoke,qakbot,mars stealer,Smoke Loader,nanocore rat,AVE_MARIA
socgholish,0.414286,0.433333,0.396970,0.511111,0.392361,0.558333,0.433333,0.450000,0.433333,0.543468,...,0.483333,0.498611,0.630946,0.673264,0.433333,0.422222,0.481944,0.455556,0.516667,0.0
mozi,0.501488,0.000000,0.498106,0.472222,0.000000,0.000000,0.000000,0.500000,0.483333,0.000000,...,0.458333,0.000000,0.000000,0.611111,0.633333,0.000000,0.444444,0.555556,0.444444,0.0
elf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.527778,0.000000,0.481481,...,0.000000,0.477778,0.000000,0.500000,0.000000,0.000000,0.000000,0.472222,0.000000,0.0
mirai,0.546280,0.000000,0.464773,0.000000,0.448611,0.487500,0.600000,0.483333,0.466667,0.437037,...,0.441667,0.000000,0.000000,0.455556,0.466667,0.455556,0.575000,0.522222,0.459722,0.0
remcos,0.539683,0.551736,0.505051,0.524306,0.488889,0.430556,0.000000,0.000000,0.525694,0.518519,...,0.430556,0.511111,0.515278,0.444444,0.577778,0.444444,0.574653,0.545139,0.437500,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
lucidacloud limited,0.463656,0.417544,0.476874,0.479450,0.481635,0.451754,0.501754,0.763114,0.417544,0.442485,...,0.451754,0.435088,0.585903,0.552577,0.417544,0.479532,0.514274,0.559642,0.559904,0.0
private layer inc,0.467787,0.419608,0.532965,0.483576,0.666208,0.578074,0.505882,0.436275,0.419608,0.672179,...,0.578400,0.596989,0.505852,0.483660,0.419608,0.483660,0.759519,0.570156,0.663457,0.0
crimsonrat,0.576190,0.433333,0.524242,0.686041,0.551736,0.631868,0.533333,0.450000,0.628776,0.461111,...,0.408333,0.524653,0.527962,0.505252,0.533333,0.511111,0.664735,0.546528,0.756507,0.0
cantv servicios venezuela,0.455233,0.493333,0.595137,0.540000,0.511110,0.608333,0.493332,0.430000,0.493291,0.585185,...,0.498333,0.613116,0.653313,0.471065,0.493292,0.539973,0.662201,0.579769,0.662212,0.0


In [56]:
def search_similarity(df, alpha):
    
    dataframe = pd.DataFrame()
    
    indexes = list(df.index)
    columns = list(df.columns)
    
    for i in range(len(indexes)):
        for j in range(len(columns)):
            
            x = indexes[i]
            y = columns[j]
                        
            element = df.at[x, y]
            
            if((element > alpha)):
#                print(f'x {x} and y: {y} and elem : {element}')
                data = {'malware_OR_alias': [y], 'tag': [x]}
                temp = pd.DataFrame(data)
                
                dataframe = pd.concat([dataframe, temp], ignore_index= True)
    
    return dataframe

In [59]:
similarity_dataframe = search_similarity(jaro_wrinkler_matrix, 0.9)
similarity_dataframe = similarity_dataframe.drop_duplicates()
similarity_dataframe.head(50)

,malware_OR_alias,tag
0,mozi,mozi
1,mirai,mirai
2,remcos,remcos
3,redline stealer,redlinestealer
4,amadey,amadey
5,xworm,xworm
6,lumma stealer,lummastealer
7,dcrat,dcrat
8,spynote,spynote
9,smokeloader,smokeloader


In [60]:
# we add similarity between tags and malwares/alias derivated from TAGS to MALWARES dataframes

In [61]:
def add_similarity_to_Malware_df(df, df_to_add):
    
    malwares = list(df.malware)
    
    alias = list(df.alias)
    
    items = df_to_add.malware_OR_alias
    
    dataframe = pd.DataFrame()
    
    for i in range(len(items)):
        
        x = df_to_add.tag[i]

        if(items[i] in malwares):
                                    
            data = {'malware': [items[i]], 'alias': [x]}
            temp = pd.DataFrame(data)
            
            dataframe = pd.concat([dataframe, temp], ignore_index= True)
        
        else:

            position = alias.index(items[i])
            
            malware = malwares[position ]
            
            data = {'malware': [malware], 'alias': [x]}
            temp = pd.DataFrame(data)
            
            dataframe = pd.concat([dataframe, temp], ignore_index= True)
            
    return dataframe   

In [19]:
similarities = add_similarity_to_Malware_df( malwares ,similarity_dataframe )
similarities

,malware,alias
0,FAKEUPDATES,SocGholish
1,Mozi,Mozi
2,Mirai,mirai
3,Remcos,remcos
4,RedLine Stealer,RedLineStealer
...,...,...
163,Kimsuky,Kimusky
164,RisePro,RiseProStealer
165,Cerberus,Cerberus
166,Crimson RAT,CrimsonRAT


In [20]:
malwares = pd.concat([ malwares , similarities], ignore_index= True)
malwares = malwares.drop_duplicates()
malwares = malwares.reset_index()
malwares

,index,malware,alias
0,0,Cobalt Strike,Agentemis
1,1,Nanocore RAT,Nancrat
2,2,Remcos,RemcosRAT
3,3,Loki Password Stealer (PWS),Burkina
4,4,DCRat,DarkCrystal RAT
...,...,...,...
264,411,Kimsuky,Kimusky
265,412,RisePro,RiseProStealer
266,413,Cerberus,Cerberus
267,414,Crimson RAT,CrimsonRAT


In [21]:
malwares = malwares.drop('index', axis=1)
malwares

,malware,alias
0,Cobalt Strike,Agentemis
1,Nanocore RAT,Nancrat
2,Remcos,RemcosRAT
3,Loki Password Stealer (PWS),Burkina
4,DCRat,DarkCrystal RAT
...,...,...
264,Kimsuky,Kimusky
265,RisePro,RiseProStealer
266,Cerberus,Cerberus
267,Crimson RAT,CrimsonRAT


### ENTRIES - Data Enrichment

In [22]:
copy_tags = tags.copy()
copy_tags

,ID_ENTRY,tag,source
0,2704827,SocGholish,URL Haus
1,2704829,Mozi,URL Haus
2,2704830,Mozi,URL Haus
3,2704831,Mozi,URL Haus
4,2704832,Mozi,URL Haus
...,...,...,...
40137,1163859,RAT,Threat Fox
40138,1163860,RAT,Threat Fox
40139,1163860,NanoCore,Threat Fox
40140,1163861,RemcosRAT,Threat Fox


In [23]:
def derive_malware(malwares, copy_tags):
    
    copy_tags ['malware'] = np.nan
    
    for i in range(len(copy_tags)):
        
        tag = copy_tags.tag[i]
        
        alias = list( malwares.alias )
        
        if( tag in alias):
        
            position = alias.index(tag)
            malware = malwares.malware[ position ]
            copy_tags.at[i, 'malware'] = malware
    
    return copy_tags

In [24]:
derive_copy_tags = derive_malware(malwares, copy_tags)
derive_copy_tags.tail(50)

,ID_ENTRY,tag,source,malware
40092,1163833,Stealc,Threat Fox,Stealc
40093,1163834,CobaltStrike,Threat Fox,Cobalt Strike
40094,1163835,CobaltStrike,Threat Fox,Cobalt Strike
40095,1163836,CobaltStrike,Threat Fox,Cobalt Strike
40096,1163837,stealer,Threat Fox,Mars Stealer
40097,1163837,Lumma,Threat Fox,Lumma Stealer
40098,1163838,Lumma,Threat Fox,Lumma Stealer
40099,1163838,stealer,Threat Fox,Mars Stealer
40100,1163839,Lumma,Threat Fox,Lumma Stealer
40101,1163839,stealer,Threat Fox,Mars Stealer


In [25]:
copy_entries = df.copy()
copy_entries

,ID_ENTRY,first_seen,ioc,status,last_seen,threat_type,reference,reporter,source,ioc_type,malware,hostname,as_number,as_name,country
0,2711900.0,2023-09-15 15:17:09,http://1.32.53.97:45188/.i,online,2023-09-15 15:17:09,payload_delivery,https://urlhaus.abuse.ch/url/2711900/,geenensp,URL Haus,url,NaN,NaN,NaN,NaN,NaN
1,2711899.0,2023-09-15 15:11:08,https://kld.workout.oystergardener.net/editContent,offline,2023-09-15 15:11:08,payload_delivery,https://urlhaus.abuse.ch/url/2711899/,Cryptolaemus1,URL Haus,url,NaN,NaN,NaN,NaN,NaN
2,2711898.0,2023-09-15 15:03:04,http://178.141.204.195:38244/Mozi.m,offline,2023-09-15 15:03:04,payload_delivery,https://urlhaus.abuse.ch/url/2711898/,Gandylyan1,URL Haus,url,NaN,NaN,NaN,NaN,NaN
3,2711897.0,2023-09-15 13:35:07,https://github.com/hoter666/8888/blob/main/Banana.exe,online,2023-09-15 15:04:29,payload_delivery,https://urlhaus.abuse.ch/url/2711897/,Casperinous,URL Haus,url,NaN,NaN,NaN,NaN,NaN
4,2711896.0,2023-09-15 13:33:07,http://mail.treeoflifeadventures.com/wp-content/plugins/70d5e28f51c1438d94e3e6dc84b95311/xt/mmd/shell/UMLrjk2KC4fpJg5.exe,offline,2023-09-15 13:56:00,payload_delivery,https://urlhaus.abuse.ch/url/2711896/,James_inthe_box,URL Haus,url,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15580,NaN,2023-08-24 23:29:43,41.97.47.7:443,offline,2023-08-25,NaN,NaN,NaN,Feodo Tracker,ip:port,QakBot,NaN,36947.0,ALGTEL-AS,ALGERIA
15581,NaN,2023-08-25 01:29:36,92.9.44.234:2222,offline,2023-08-25,NaN,NaN,NaN,Feodo Tracker,ip:port,QakBot,host-92-9-44-234.as13285.net,13285.0,OPALTELECOM-AS TalkTalk Communications Limited,GREAT BRITAIN
15582,NaN,2023-08-25 10:08:52,78.152.198.132:443,offline,2023-08-25,NaN,NaN,NaN,Feodo Tracker,ip:port,QakBot,NaN,15502.0,VODAFONE-IRELAND-ASN,IRELAND
15583,NaN,2023-08-25 11:24:40,113.193.95.229:443,offline,2023-08-25,NaN,NaN,NaN,Feodo Tracker,ip:port,QakBot,NaN,45528.0,TIKONAIN-AS Tikona Infinet Ltd.,INDIA


In [26]:
tag_frequencies = derive_copy_tags.groupby(['ID_ENTRY', 'source', 'malware']).size().reset_index(name='count')

# Trovare il tag più frequente per ogni item con il majority voting
majority_tags = tag_frequencies.loc[tag_frequencies.groupby(['ID_ENTRY', 'source'])['count'].idxmax()]

# Nel caso di parità, scegliere casualmente uno dei tag più frequenti
random_majority_tags = majority_tags.groupby(['ID_ENTRY', 'source']).apply(lambda group: group.sample(1)).reset_index(drop=True)

# Unire il risultato al dataset A sulla base della colonna "item"
result_df = pd.merge(copy_entries, random_majority_tags, on=['ID_ENTRY', 'source'])

# Rinominare la colonna risultante
result_df.rename(columns={'malware': 'malware'}, inplace=True)

In [27]:
result_df

,ID_ENTRY,first_seen,ioc,status,last_seen,threat_type,reference,reporter,source,ioc_type,malware_x,hostname,as_number,as_name,country,malware_y,count
0,2711899.0,2023-09-15 15:11:08,https://kld.workout.oystergardener.net/editContent,offline,2023-09-15 15:11:08,payload_delivery,https://urlhaus.abuse.ch/url/2711899/,Cryptolaemus1,URL Haus,url,NaN,NaN,NaN,NaN,NaN,FAKEUPDATES,1
1,2711898.0,2023-09-15 15:03:04,http://178.141.204.195:38244/Mozi.m,offline,2023-09-15 15:03:04,payload_delivery,https://urlhaus.abuse.ch/url/2711898/,Gandylyan1,URL Haus,url,NaN,NaN,NaN,NaN,NaN,Mozi,1
2,2711896.0,2023-09-15 13:33:07,http://mail.treeoflifeadventures.com/wp-content/plugins/70d5e28f51c1438d94e3e6dc84b95311/xt/mmd/shell/UMLrjk2KC4fpJg5.exe,offline,2023-09-15 13:56:00,payload_delivery,https://urlhaus.abuse.ch/url/2711896/,James_inthe_box,URL Haus,url,NaN,NaN,NaN,NaN,NaN,Nanocore RAT,1
3,2711895.0,2023-09-15 13:18:06,http://galandskiyher2.com/downloads/toolspub3.exe,online,2023-09-15 14:54:39,payload_delivery,https://urlhaus.abuse.ch/url/2711895/,vxvault,URL Haus,url,NaN,NaN,NaN,NaN,NaN,Stealc,1
4,2711893.0,2023-09-15 13:07:06,http://galandskiyher2.com/downloads/toolspub2.exe,online,2023-09-15 15:10:11,payload_delivery,https://urlhaus.abuse.ch/url/2711893/,vxvault,URL Haus,url,NaN,NaN,NaN,NaN,NaN,Stealc,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13059,1155704.0,2023-09-08 19:25:17 UTC,http://berkatamankimsedurduramaz.ml/,NaN,2023-09-08 19:25:17 UTC,botnet,https://malpedia.caad.fkie.fraunhofer.de/details/apk.alien,myonium1,Threat Fox,url,Alien,NaN,NaN,NaN,NaN,Alien,1
13060,1155701.0,2023-09-08 14:00:16 UTC,41.216.188.29:5200,NaN,2023-09-08 14:00:16 UTC,botnet,https://malpedia.caad.fkie.fraunhofer.de/details/win.ave_maria,abuse_ch,Threat Fox,ip:port,Ave Maria,NaN,NaN,NaN,NaN,Ave Maria,1
13061,1155700.0,2023-09-08 13:35:13 UTC,193.31.28.123:4444,NaN,2023-09-08 13:35:13 UTC,botnet,https://malpedia.caad.fkie.fraunhofer.de/details/win.meterpreter,abuse_ch,Threat Fox,ip:port,Meterpreter,NaN,NaN,NaN,NaN,Meterpreter,1
13062,1155698.0,2023-09-08 13:33:51 UTC,162.33.179.240:433,NaN,2023-09-08 13:33:51 UTC,botnet,https://malpedia.caad.fkie.fraunhofer.de/details/win.icedid,abuse_ch,Threat Fox,ip:port,IcedID,NaN,NaN,NaN,NaN,IcedID,1


In [28]:
result_df['malware_x'].fillna(result_df['malware_y'], inplace=True)

# Eliminare la colonna 'malware_y'
result_df.drop(columns=['malware_y'], inplace=True)

# Rinominare 'malware_x' in 'malware'
result_df.rename(columns={'malware_x': 'malware'}, inplace=True)


In [29]:
result_df

,ID_ENTRY,first_seen,ioc,status,last_seen,threat_type,reference,reporter,source,ioc_type,malware,hostname,as_number,as_name,country,count
0,2711899.0,2023-09-15 15:11:08,https://kld.workout.oystergardener.net/editContent,offline,2023-09-15 15:11:08,payload_delivery,https://urlhaus.abuse.ch/url/2711899/,Cryptolaemus1,URL Haus,url,FAKEUPDATES,NaN,NaN,NaN,NaN,1
1,2711898.0,2023-09-15 15:03:04,http://178.141.204.195:38244/Mozi.m,offline,2023-09-15 15:03:04,payload_delivery,https://urlhaus.abuse.ch/url/2711898/,Gandylyan1,URL Haus,url,Mozi,NaN,NaN,NaN,NaN,1
2,2711896.0,2023-09-15 13:33:07,http://mail.treeoflifeadventures.com/wp-content/plugins/70d5e28f51c1438d94e3e6dc84b95311/xt/mmd/shell/UMLrjk2KC4fpJg5.exe,offline,2023-09-15 13:56:00,payload_delivery,https://urlhaus.abuse.ch/url/2711896/,James_inthe_box,URL Haus,url,Nanocore RAT,NaN,NaN,NaN,NaN,1
3,2711895.0,2023-09-15 13:18:06,http://galandskiyher2.com/downloads/toolspub3.exe,online,2023-09-15 14:54:39,payload_delivery,https://urlhaus.abuse.ch/url/2711895/,vxvault,URL Haus,url,Stealc,NaN,NaN,NaN,NaN,1
4,2711893.0,2023-09-15 13:07:06,http://galandskiyher2.com/downloads/toolspub2.exe,online,2023-09-15 15:10:11,payload_delivery,https://urlhaus.abuse.ch/url/2711893/,vxvault,URL Haus,url,Stealc,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13059,1155704.0,2023-09-08 19:25:17 UTC,http://berkatamankimsedurduramaz.ml/,NaN,2023-09-08 19:25:17 UTC,botnet,https://malpedia.caad.fkie.fraunhofer.de/details/apk.alien,myonium1,Threat Fox,url,Alien,NaN,NaN,NaN,NaN,1
13060,1155701.0,2023-09-08 14:00:16 UTC,41.216.188.29:5200,NaN,2023-09-08 14:00:16 UTC,botnet,https://malpedia.caad.fkie.fraunhofer.de/details/win.ave_maria,abuse_ch,Threat Fox,ip:port,Ave Maria,NaN,NaN,NaN,NaN,1
13061,1155700.0,2023-09-08 13:35:13 UTC,193.31.28.123:4444,NaN,2023-09-08 13:35:13 UTC,botnet,https://malpedia.caad.fkie.fraunhofer.de/details/win.meterpreter,abuse_ch,Threat Fox,ip:port,Meterpreter,NaN,NaN,NaN,NaN,1
13062,1155698.0,2023-09-08 13:33:51 UTC,162.33.179.240:433,NaN,2023-09-08 13:33:51 UTC,botnet,https://malpedia.caad.fkie.fraunhofer.de/details/win.icedid,abuse_ch,Threat Fox,ip:port,IcedID,NaN,NaN,NaN,NaN,1


In [30]:
dataframes_dict = {}

# Add the DataFrames to the dictionary with keys
dataframes_dict['Entries'] = result_df
dataframes_dict['Tags'] = tags
dataframes_dict['Malwares'] = malwares

In [31]:
save_directory = "C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\DataEnrichment\\"
for key, df in dataframes_dict.items():
    file_name = f"{key}.csv"
    file_path = save_directory + file_name
    df.to_csv(file_path, index=False)